# SCE-TTS: 음성합성 데모

이 문서는 SCE-TTS 프로젝트의 음성 합성 데모입니다.

이 데모에 대한 더 자세한 정보는 아래 링크에서 확인하실 수 있습니다.  
https://sce-tts.github.io/

## 1. 구글 드라이브 마운트

음성합성을 위해 학습한 모델이 있는 구글 드라이브를 마운트합니다.  
마운트할 구글 드라이브 내에 다음 파일들이 존재하는지 꼭 확인해주세요.

- `/Colab Notebooks/data/glowtts-v2/model_file.pth.tar`
- `/Colab Notebooks/data/glowtts-v2/config.json`
- `/Colab Notebooks/data/hifigan-v2/model_file.pth.tar`
- `/Colab Notebooks/data/hifigan-v2/config.json`


(존재하지 않는다면, [glowtts-v2.zip](https://drive.google.com/file/d/1DMKLdfZ_gzc_z0qDod6_G8fEXj0zCHvC/view?usp=sharing), [hifigan-v2.zip](https://drive.google.com/file/d/1vRxp1RH-U7gSzWgyxnKY4h_7pB3tjPmU/view?usp=sharing)을 내려받아 준비해주세요.)

만약 아래에 `Enter your authorization code:`과 같은 메시지가 출력될 경우,  
같이 출력된 링크에 접속하여, 마운트할 구글 계정을 선택하신 후, 인증 코드를 복사하여 입력해주세요.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 2. 필수 라이브러리 및 함수 불러오기

실행에 필요한 라이브러리 및 함수를 불러옵니다.

이 과정은 약 10분 정도 소요될 수 있습니다.

In [ ]:
import os
import sys
from pathlib import Path

In [ ]:
%cd /content
!git clone --depth 1 https://github.com/sce-tts/TTS.git -b sce-tts
!git clone --depth 1 https://github.com/sce-tts/g2pK.git
%cd /content/TTS
!pip install -q --no-cache-dir -e .
%cd /content/g2pK
!pip install -q --no-cache-dir "konlpy" "jamo" "nltk" "python-mecab-ko"
!pip install -q --no-cache-dir -e .

/content
Cloning into 'TTS'...
remote: Enumerating objects: 447, done.
remote: Counting objects: 100% (447/447), done.
remote: Compressing objects: 100% (413/413), done.
remote: Total 447 (delta 56), reused 222 (delta 22), pack-reused 0
Receiving objects: 100% (447/447), 13.77 MiB | 25.63 MiB/s, done.
Resolving deltas: 100% (56/56), done.
Cloning into 'g2pK'...
remote: Enumerating objects: 20, done.
remote: Counting objects: 100% (20/20), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 20 (delta 0), reused 14 (delta 0), pack-reused 0
Unpacking objects: 100% (20/20), done.
/content/TTS
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 1.3 MB 5.0 MB/s 
     |████████████████████████████████| 20.1 MB 1.1 MB/s 
     |████████████████████████████████| 125 kB 55.3 MB/s 
     |████████████████████████████████| 69 kB 55.2 MB/s 
     |█████████████████████████

In [ ]:
%cd /content/g2pK
import g2pk
g2p = g2pk.G2p()

/content/g2pK
[nltk_data] Downloading package cmudict to /root/nltk_data...
[nltk_data]   Unzipping corpora/cmudict.zip.


In [ ]:
%cd /content/TTS
import re
import sys
from unicodedata import normalize
import IPython

from TTS.utils.synthesizer import Synthesizer

def normalize_text(text):
    text = text.strip()

    for c in ",;:":
        text = text.replace(c, ".")
    text = remove_duplicated_punctuations(text)

    text = jamo_text(text)

    text = g2p.idioms(text)
    text = g2pk.english.convert_eng(text, g2p.cmu)
    text = g2pk.utils.annotate(text, g2p.mecab)
    text = g2pk.numerals.convert_num(text)
    text = re.sub("/[PJEB]", "", text)

    text = alphabet_text(text)

    # remove unreadable characters
    text = normalize("NFD", text)
    text = "".join(c for c in text if c in symbols)
    text = normalize("NFC", text)

    text = text.strip()
    if len(text) == 0:
        return ""

    # only single punctuation
    if text in '.!?':
        return punctuation_text(text)

    # append punctuation if there is no punctuation at the end of the text
    if text[-1] not in '.!?':
        text += '.'

    return text


def remove_duplicated_punctuations(text):
    text = re.sub(r"[.?!]+\?", "?", text)
    text = re.sub(r"[.?!]+!", "!", text)
    text = re.sub(r"[.?!]+\.", ".", text)
    return text


def split_text(text):
    text = remove_duplicated_punctuations(text)

    texts = []
    for subtext in re.findall(r'[^.!?\n]*[.!?\n]', text):
        texts.append(subtext.strip())

    return texts


def alphabet_text(text):
    text = re.sub(r"(a|A)", "에이", text)
    text = re.sub(r"(b|B)", "비", text)
    text = re.sub(r"(c|C)", "씨", text)
    text = re.sub(r"(d|D)", "디", text)
    text = re.sub(r"(e|E)", "이", text)
    text = re.sub(r"(f|F)", "에프", text)
    text = re.sub(r"(g|G)", "쥐", text)
    text = re.sub(r"(h|H)", "에이치", text)
    text = re.sub(r"(i|I)", "아이", text)
    text = re.sub(r"(j|J)", "제이", text)
    text = re.sub(r"(k|K)", "케이", text)
    text = re.sub(r"(l|L)", "엘", text)
    text = re.sub(r"(m|M)", "엠", text)
    text = re.sub(r"(n|N)", "엔", text)
    text = re.sub(r"(o|O)", "오", text)
    text = re.sub(r"(p|P)", "피", text)
    text = re.sub(r"(q|Q)", "큐", text)
    text = re.sub(r"(r|R)", "알", text)
    text = re.sub(r"(s|S)", "에스", text)
    text = re.sub(r"(t|T)", "티", text)
    text = re.sub(r"(u|U)", "유", text)
    text = re.sub(r"(v|V)", "브이", text)
    text = re.sub(r"(w|W)", "더블유", text)
    text = re.sub(r"(x|X)", "엑스", text)
    text = re.sub(r"(y|Y)", "와이", text)
    text = re.sub(r"(z|Z)", "지", text)

    return text


def punctuation_text(text):
    # 문장부호
    text = re.sub(r"!", "느낌표", text)
    text = re.sub(r"\?", "물음표", text)
    text = re.sub(r"\.", "마침표", text)

    return text


def jamo_text(text):
    # 기본 자모음
    text = re.sub(r"ㄱ", "기역", text)
    text = re.sub(r"ㄴ", "니은", text)
    text = re.sub(r"ㄷ", "디귿", text)
    text = re.sub(r"ㄹ", "리을", text)
    text = re.sub(r"ㅁ", "미음", text)
    text = re.sub(r"ㅂ", "비읍", text)
    text = re.sub(r"ㅅ", "시옷", text)
    text = re.sub(r"ㅇ", "이응", text)
    text = re.sub(r"ㅈ", "지읒", text)
    text = re.sub(r"ㅊ", "치읓", text)
    text = re.sub(r"ㅋ", "키읔", text)
    text = re.sub(r"ㅌ", "티읕", text)
    text = re.sub(r"ㅍ", "피읖", text)
    text = re.sub(r"ㅎ", "히읗", text)
    text = re.sub(r"ㄲ", "쌍기역", text)
    text = re.sub(r"ㄸ", "쌍디귿", text)
    text = re.sub(r"ㅃ", "쌍비읍", text)
    text = re.sub(r"ㅆ", "쌍시옷", text)
    text = re.sub(r"ㅉ", "쌍지읒", text)
    text = re.sub(r"ㄳ", "기역시옷", text)
    text = re.sub(r"ㄵ", "니은지읒", text)
    text = re.sub(r"ㄶ", "니은히읗", text)
    text = re.sub(r"ㄺ", "리을기역", text)
    text = re.sub(r"ㄻ", "리을미음", text)
    text = re.sub(r"ㄼ", "리을비읍", text)
    text = re.sub(r"ㄽ", "리을시옷", text)
    text = re.sub(r"ㄾ", "리을티읕", text)
    text = re.sub(r"ㄿ", "리을피읍", text)
    text = re.sub(r"ㅀ", "리을히읗", text)
    text = re.sub(r"ㅄ", "비읍시옷", text)
    text = re.sub(r"ㅏ", "아", text)
    text = re.sub(r"ㅑ", "야", text)
    text = re.sub(r"ㅓ", "어", text)
    text = re.sub(r"ㅕ", "여", text)
    text = re.sub(r"ㅗ", "오", text)
    text = re.sub(r"ㅛ", "요", text)
    text = re.sub(r"ㅜ", "우", text)
    text = re.sub(r"ㅠ", "유", text)
    text = re.sub(r"ㅡ", "으", text)
    text = re.sub(r"ㅣ", "이", text)
    text = re.sub(r"ㅐ", "애", text)
    text = re.sub(r"ㅒ", "얘", text)
    text = re.sub(r"ㅔ", "에", text)
    text = re.sub(r"ㅖ", "예", text)
    text = re.sub(r"ㅘ", "와", text)
    text = re.sub(r"ㅙ", "왜", text)
    text = re.sub(r"ㅚ", "외", text)
    text = re.sub(r"ㅝ", "워", text)
    text = re.sub(r"ㅞ", "웨", text)
    text = re.sub(r"ㅟ", "위", text)
    text = re.sub(r"ㅢ", "의", text)

    return text


def normalize_multiline_text(long_text):
    texts = split_text(long_text)
    normalized_texts = [normalize_text(text).strip() for text in texts]
    return [text for text in normalized_texts if len(text) > 0]

def synthesize(text):
    wavs = synthesizer.tts(text, None, None)
    return wavs

/content/TTS


## 3. 학습한 모델 불러오기

학습한 Glow-TTS와 HiFi-GAN 모델을 불러옵니다.

만약 다른 체크포인트에서 불러오시려면 아래 코드에서 경로를 아래와 같이 적절하게 수정합니다.

```python
synthesizer = Synthesizer(
    "/content/drive/My Drive/Colab Notebooks/data/glowtts-v2/glowtts-v2-May-31-2021_08+17AM-d897f2e/best_model.pth.tar",
    "/content/drive/My Drive/Colab Notebooks/data/glowtts-v2/glowtts-v2-May-31-2021_08+17AM-d897f2e/config.json",
    None,
    "/content/drive/My Drive/Colab Notebooks/data/hifigan-v2/hifigan-v2-May-31-2021_08+26AM-d897f2e/checkpoint_300000.pth.tar",
    "/content/drive/My Drive/Colab Notebooks/data/hifigan-v2/hifigan-v2-May-31-2021_08+26AM-d897f2e/config.json",
    None,
    None,
    False,
)
```

In [ ]:
synthesizer = Synthesizer(
    "/content/drive/MyDrive/Colab Notebooks/suyeon/glowtts-v2/glowtts-v2-May-29-2022_11+22PM-3aa165a/best_model.pth.tar",
    "/content/drive/MyDrive/Colab Notebooks/suyeon/glowtts-v2/glowtts-v2-May-29-2022_11+22PM-3aa165a/config.json",
    None,
    "/content/drive/MyDrive/Colab Notebooks/suyeon/hifigan-v2/hifigan-v2-May-28-2022_11+53AM-3aa165a/best_model.pth.tar",
    "/content/drive/MyDrive/Colab Notebooks/suyeon/hifigan-v2/hifigan-v2-May-28-2022_11+53AM-3aa165a/config.json",
    None,
    None,
    False,
)
symbols = synthesizer.tts_config.characters.characters

 > Using model: glow_tts
 > Generator Model: hifigan_generator
Removing weight norm...


## 4. 음성 합성

실제 음성 합성을 수행합니다.

`long_text`의 값을 변경하여 다른 문장의 합성도 시도해보실 수 있습니다.

In [ ]:
import time

In [ ]:
texts = """
두 모델의 최종 결과물에 영향을 주지 않습니다.
"""
for text in normalize_multiline_text(texts):
    wav = synthesizer.tts(text, None, None)
    IPython.display.display(IPython.display.Audio(wav, rate=22050))  

 > Text splitted to sentences.
['두 모델의 최종 결과물에 영향을 주지 않습니다.']
 > Processing time: 0.9033613204956055
 > Real-time factor: 0.2281372218816211


In [ ]:
texts = """
흥부전은 작자·연대 미상의 고전소설입니다. 조선 후기 소설로 흥보전,박흥보전,놀부전,흥보가 등으로 불려집니다. 흥부전의 줄거리는 형인 놀부가 부모님의 유산을 독차지 하고 동생인 흥부를 집에서 쫓아 냅니다. 돈 한푼 없이 쫓겨난 흥부는 부인과 아이들과 가난하게 지낼 수 밖에 없었고, 놀부는 물려받은 부모님의 재산으로 호화스럽게 살고 있었습니다. 어느 날 흥부는 형인 놀부네 집에 가서 돈과 식량이 없어서 그러니 형인 놀부에게 한 푼만 달라고 찾아갔는데, 놀부의 아내가 밥풀이 묻은 주걱으로 흥부의 뺨을 세게 치면서 불쌍한 흥부를 내쫓았습니다. 불쌍한 흥부는 뺨이 퉁퉁 부은 상태로 자신의 초라한 오두막으로 돌아왔습니다. 어느 봄날, 제비가 흥부네 집 마당에 다리를 다친 채로 쓰러져 있었습니다. 흥부는 조그만 제비가 가여워 다리를 고쳐주었더니 다음해 봄날 제비가 날라와서 흥부에게 박씨를 주고 갔습니다. 흥부와 흥부 아내는 제비가 물어다 준 조그만 박씨를 집 마당 한 켠에 심었는데, 그 해 가을 박이 집이 무너질 정도로 주렁주렁 열려있어서, 박을 켜는 톱을 빌려와 박을 켜기 시작했습니다. 첫 번째 박을 켜자 값을 가늠할 수 없는 약재와, 귀한 술들이 쏟아져 나왔습니다. 이에 너무 놀란 흥부 가족은 이어서 두 번째 박을 열어 보았습니다. 두 번째 박을 켜자 박 안에서 무수히 많은 각종 가구와 귀한 책들이 쏟아져 나왔고 세 번째 박에서는 집을 짓는 목수와 산더미 같은 곡식들이 나왔습니다. 그리고 다른 박들 에서는 어마어마한 돈과 휘황찬란한 비단, 베, 모시 등의 의복류가 박 속에서 나왔고 흥부와 흥부의 아내 그리고 흥부의 자식들은 기쁨을 감추지 못하고 환호하고 행복해 했습니다. 이 이야기를 들은 놀부는 동생 흥부가 엄청난 행운을 얻었다는 소식을 듣고 흥부에게 비결을 물어봤습니다. 착한 흥부는 형 놀부에게 소상히 알려주었고 놀부는 단걸음에 집으로 돌아왔습니다. 다음 해 봄 놀부는 제비를 잡고 일부러 다리를 부러뜨린 뒤 고쳐 주고는 그토록 기다리던 박씨를 받게 됩니다. 그 해 가을, 놀부네 지붕에는 흥부네 지붕에 열렸던 박만큼이나 커다란 박들이 주렁주렁 열렸습니다. 놀부 내외는 커다란 박을 보며 흥부만큼이나 큰 재물들을 얻을 수 있다는 기쁜 마음을 안고서 박을 켜기 시작했습니다. 하지만 놀부 부부가 켠 박에선 수 많은 괴한들이 나와서 놀부네 재산들을 가져가고 다른 박에선 괴물들이 나타나 놀부의 집을 부시고 사라졌습니다. 그리고 다음 박에선 온갖 더러운 오물들이 넘쳐나서 놀부의 집은 완전 아수라장이 되었습니다. 하루아침만에 놀부는 가난뱅이가 되어버렸습니다. 놀부는 끝없는 욕심을 부리다가 이렇게 벌을 받게 되었죠.
비록 형인 놀부에겐 천대받았던 흥부였지만, 이 소식을 들은 심성이 착한 흥부는 내색하지 않고 자신의 재산을 놀부에게 나눠주고 놀부를 지성으로 섬겨서 함께 행복하게 살았다고 합니다.
놀부처럼 너무 욕심을 부려서 재물을 탐내기 보다는 흥부처럼 용서하고 베푸는 마음을 가질 수 있도록 노력하고 착하게 살면 흥부처럼 언젠간 행운의 제비가 찾아오겠죠?
"""
for text in normalize_multiline_text(texts):
    wav = synthesizer.tts(text, None, None)
    t = round(len(wav) * 0.000043)+0.5
    IPython.display.display(IPython.display.Audio(wav, rate=22050, autoplay=True))  
    time.sleep(t)

In [ ]:
texts = """
두 모델의 학습은 완전히 독립적으로 수행되므로, 두 모델의 학습 순서는 최종 결과물에 영향을 주지 않습니다.
"""
for text in normalize_multiline_text(texts):
    wav = synthesizer.tts(text, None, None)
    IPython.display.display(IPython.display.Audio(wav, rate=22050))  

 > Text splitted to sentences.
['두 모델의 학습은 완전히 독립적으로 수행되므로.', '두 모델의 학습 순서는 최종 결과물에 영향을 주지 않습니다.']
 > Processing time: 3.209859848022461
 > Real-time factor: 0.3516924871248175
